# Event Stat Scraper (places all stats into array)

### Scrapes statistics of players in each event and converts it into a graph.

In [4]:
import numpy as np
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import requests

In [ ]:
from platform import python_version
python_version()


In [ ]:
event_links = ['/event/matches/1130/?series_id=2190',
 '/event/matches/800/?series_id=1953',
 '/event/matches/800/?series_id=1561',
 '/event/matches/799/?series_id=1737',
 '/event/matches/799/?series_id=1559',
 '/event/matches/558/?series_id=1094',
 '/event/matches/578/?series_id=1195',
 '/event/matches/576/?series_id=1132',
 '/event/matches/520/?series_id=1131',
 '/event/matches/372/?series_id=770',
 '/event/matches/371/?series_id=769',
 '/event/matches/370/?series_id=767',
 '/event/matches/333/?series_id=690',
 '/event/matches/324/?series_id=664',
 '/event/matches/306/?series_id=618',
 '/event/matches/291/?series_id=592']


In [ ]:
for i in range(0, len(event_links)):
    event_links[i]= "http://vlr.gg"+event_links[i]
event_links.pop(0)
event_links

In [ ]:
all_links = []
for event in event_links:
    resp = requests.get(event)
    soup = BeautifulSoup(resp.text)
    first_match_list = soup.find_all('a', class_='wf-module-item match-item mod-color mod-left mod-bg-after-red mod-first', href = True)
    second_match_list = soup.find_all('a', class_='wf-module-item match-item mod-color mod-left mod-bg-after-red', href = True)
    match_list = first_match_list + second_match_list

    link_list = []

    for match in match_list:
        link_list.append(match['href'])

    for i, link in enumerate(link_list):
        link_list[i] = "https://vlr.gg"+link_list[i]+"?game=all&tab=overview"

    all_links += link_list
all_links

In [ ]:
def clean_data(df):
    df.drop('Unnamed: 1', axis=1, inplace=True)
    df['Name'] = 'x'
    df['Team'] = 'x'

    for i in df.index:
        s = df.at[i, 'Unnamed: 0'].split()
        df.at[i,'Name'] = s[0]
        df.at[i,'Team']= s[1]
    
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.drop('+/–', axis=1, inplace=True)
    df.drop('+/–.1', axis=1, inplace=True)

    
    for i in df.index:
        for col in df.columns:
            if(isinstance(df.at[i,col], np.float64)):
                df.at[i, col] = 0
            else:
                s = df.at[i, col].split(" ")
                df.at[i,col] = s[0]
                if col == 'D':
                    df.at[i,col] = s[2]




In [ ]:
def convert_percentage(df):
    for i in df.index:
         #Cleaning KAST Percentages
        if(type(df.at[i, 'KAST'])==float):
            df.at[i, 'KAST'] = 0
        #Adjusts for if KAST was not recorded in the stats    
        elif(isinstance(df.at[i,'KAST'], np.floating)):
            df.at[i,'KAST'] = -1
        else:
            df.at[i,'KAST']= df.at[i,'KAST'].replace('%','')
        
        #Cleaning Headshot Percentages
        #Adjusts for if HS% was not recorded in the stats or player didn't headshot
        if(type(df.at[i,'HS%'])==float):
            df.at[i,'HS%'] = 0
        else:
            df.at[i,'HS%'] = df.at[i,'HS%'].replace('%','')
        
        #Cleaning Clutch Percentages
        #Adjusts for if KAST was not recorded in the stats or the player had no clutches  
        if 'CL%' in df.columns:
            if(type(df.at[i,'CL%'])==float):
                df.at[i,'CL%'] = 0
            else:
                df.at[i,'CL%'] = df.at[i,'CL%'].replace('%','')

In [ ]:
dfs = pd.read_html(link_list[0])
df = dfs[3]
df1 = dfs[2]
df.head(5)

In [ ]:
clean_data(df)
df.head(5)

In [ ]:
class Player():
    def __init__(self, arr):
        self.name = arr[9]
        self.team = arr[10]
        self.acs = [float(arr[0])]
        self.k =[float(arr[1])] 
        self.d =[float(arr[2])]  
        self.a =[float(arr[3])] 
        self.kast =[float(arr[4])] 
        self.adr =[float(arr[5])] 
        self.hsp =[float(arr[6])] 
        self.fk =[float(arr[7])] 
        self.fd =[float(arr[8])] 
        self.games = []
    
    def set_stats(self, arr):
        self.acs.append(float(arr[0]))
        self.k.append(float(arr[1]) )
        self.d.append(float(arr[2]))  
        self.a.append(float(arr[3])) 
        self.kast.append(float(arr[4])) 
        self.adr.append(float(arr[5])) 
        self.hsp.append(float(arr[6])) 
        self.fk.append(float(arr[7]))
        self.fd.append(float(arr[8])) 
    def get_name(self):
        return self.name
    
    def get_stats(self):
        return [self.name, self.team, self.acs, self.k, self.d, self.a, self.kast, self.adr, self.hsp, self.fk, self.fd]

In [ ]:
player_names =[]
player_list =[]

In [ ]:
for link in all_links:
    dfs = pd.read_html(link)
    df = dfs[2]
    df1 = dfs[3]
    clean_data(df)
    clean_data(df1)
    convert_percentage(df)
    convert_percentage(df1)
    for x, i in enumerate(df.index):
        if df.iloc[i]['Name'] in player_names:
            index = player_names.index(df.iloc[i]['Name'])
            player_list[index].set_stats(df.iloc[i].to_numpy())
        else:
            player_list.append(Player(df.iloc[i].to_numpy()))
            player_names.append(df.iloc[i]['Name'])
    for x, i in enumerate(df1.index):
        if df1.iloc[i]['Name'] in player_names:

            index = player_names.index(df1.iloc[i]['Name'])
            player_list[index].set_stats(df1.iloc[i].to_numpy())
        else:
            player_list.append(Player(df1.iloc[i].to_numpy()))
            player_names.append(df1.iloc[i]['Name'])

In [ ]:
player_list[6].__dict__